# Dectecing Forest Fires using CNN

Make sure you set up two conda envs before you run this notebook

First we need to pre-process the dataset to a sutable 100x100 dimension matrix for the neural network.
For this we make use of opencv the unprocessed data is in the ./uprocessed data folder and the processed 
data is in the ./data folder.

The data is already been proecessed for you and you are not required to run this step.

In [1]:
import cv2
import os

In [13]:
def image_processing(raw_data,data_path,height,width):
    class_labels=[]
    category_count=0
    for i in os.walk(raw_data):
        if len(i[2])>0:
            counter=0
            images=i[2]
            class_name=i[0].strip('\\')
            print(class_name)
            path=os.path.join(data_path,class_labels[category_count])
            for image in images:
                im=cv2.imread(class_name+'\\'+image)
                print(image)
                im=cv2.resize(im,(height,width))
                if not os.path.exists(path):
                    os.makedirs(path)
                cv2.imwrite(os.path.join(path,str(counter)+'.jpg'),im)
                counter+=1
            category_count+=1
        else:
            number_of_classes=len(i[1])
            print(number_of_classes,i[1])
            class_labels=i[1][:]

<p style="color:red"> You can skip running the below block and if you did you need to sort out the images in the following folder structure</p>
data=>(test,train,valid)=>(fire,forest)

In [14]:
height = 100
width = 100
raw_data = 'unprocessed-data'
data_path = 'data'
image_processing(raw_data, data_path, height, width)

2 ['forest', 'forest fire']
unprocessed-data\forest
1. deforestation-what-wwf-doing-introhi_107217.jpg
10. 31105585103_4c32392ac1_k.jpg
100. forest.jpg
101. 220px-forrest_in_northern_bohemia_-_les_na_severu_%c4%8cech.jpg
102. merlin_141448098_8c5466da-3709-48a6-a9c2-dabda74d5bf7-articlelarge.jpg
103. depositphotos_73982563_m-2015.jpg
104. forest_fire_fighter.jpg
105. latvian_forest_tomes_pagasts%2c_%c4%b6eguma_novads%2c_latvia.jpg
106. theforestscreenshot_08.jpg
107. forest-area-banner.jpg
108. 559672101517195076987621071193712n.jpg
109. gp03a12.jpg
11. estonian_forest_reasons_to_visit_sven_zacek.jpg
110. 180321155319_1_540x360.jpg
111. forest%20-%20hm44%20apr-2012%20trees%20.jpg
112. fseprd576671_tn.jpg
113. 1841517.jpg
115. forests-k6--621x414@livemint.jpg
116. hy_forestsciences_hires_0.jpg
117. longleafpineforestnorthcarolina.jpg
118. forest.jpg
119. the-forest.jpg
12. 5325.jpg
120. semievergreen.jpg
122. 772526270ea45caa5d837f9dc053fe3f3e1c38ca.jpg
123. climate-change-forests-squar

Here is a few sample images to see how the data looks like.
<br><img src="./data/test/fire/68.jpg" style="display:inline"/><img src="./data/test/fire/88.jpg" style="display:inline"/><img src="./data/test/fire/107.jpg" style="display:inline"/><br>
The dataset contains 600+ images in jpg format.

### Step 1
Lets start with importing our data and genrating some insights on it. 
The data is divided into train, test, validation sets with a split of aproximatly 60%, 20%, 20%.

In [1]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 133)
    return files, targets

train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')

class_names = [item[11:-1] for item in sorted(glob("data/train/*/"))]

print('There are %d total categories.' % len(class_names))
print('There are %s total images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training images.' % len(train_files))
print('There are %d validation images.' % len(valid_files))
print('There are %d test images.'% len(test_files))

Using TensorFlow backend.


There are 2 total categories.
There are 606 total images.

There are 386 training images.
There are 110 validation images.
There are 110 test images.


### Step 2
Next lets convert our 100x100x3 dimension image to a 1x100x100x3 tensor.

In [3]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path, target_size=(100, 100))
    x = image.img_to_array(img)
    # converting to a tensor of dimensions (1,100,100,3)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [4]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

100%|██████████████████████████████████████████████████████████████████████████████| 110/110 [00:00<00:00, 1133.33it/s]


### Step 3
Lets start builing our CNN model with keras

In [20]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

In [26]:
model = Sequential()
model.add(Conv2D(32,(3, 3),activation='relu',input_shape=train_tensors[0].shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(133, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 22, 22, 128)       73856     
__________

In [27]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Step 3
In this step we train the model and store checkpoints.

In [29]:
from keras.callbacks import ModelCheckpoint  

epochs = 5
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=20, callbacks=[checkpointer], verbose=1)

Train on 386 samples, validate on 110 samples
Epoch 1/5
386/386 [==============================] - 17s - loss: 0.7203 - acc: 0.7565 - val_loss: 0.3649 - val_acc: 0.8545
Epoch 2/5
386/386 [==============================] - 19s - loss: 0.5092 - acc: 0.7927 - val_loss: 0.6870 - val_acc: 0.8909
Epoch 3/5
386/386 [==============================] - 20s - loss: 0.4288 - acc: 0.8316 - val_loss: 0.3713 - val_acc: 0.8636
Epoch 4/5
386/386 [==============================] - 19s - loss: 0.3253 - acc: 0.8834 - val_loss: 0.2360 - val_acc: 0.8727
Epoch 5/5
386/386 [==============================] - 21s - loss: 0.4226 - acc: 0.8808 - val_loss: 0.1667 - val_acc: 0.9364


In [30]:
print(train_tensors.shape)

(386, 100, 100, 3)


In [31]:
predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]
test_accuracy = 100*np.sum(np.array(predictions)==np.argmax(test_targets, axis=1))/len(predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 94.5455%


### Step 4 
Testing the with some images.
<img src="fire.jpg"/>

In [47]:
img = cv2.imread('./fire.jpg')
img=cv2.resize(img,(100,100))
img = np.expand_dims(img, axis=0)

In [48]:
if model.predict(img)[0][1] == 1.0:
    print("Normal")
elif model.predict(img)[0][0] == 1.0:
    print("Fire")

Fire


### Step6 
Exporting the model to model.json

In [85]:
model_json = model.to_json()

In [87]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


### Step 7
Once you are done with this notebook you can try out the model with video stream by running test.py file.
To run test.py you need to enable the second enviorment or you may encounter errors with OpenCV.